In [7]:
import json
import importlib
from nba_api.stats import endpoints

# Sample parameters for testing
SAMPLE_PARAMS = {
    'game_id': '0022300001',
    'player_id': '2544',  # LeBron James
    'team_id': '1610612747',  # Lakers
    'season': '2023-24',
    'season_type': 'Regular Season'
}

def get_unknown_endpoints():
    """Load all endpoints marked as 'unknown' priority from the config file"""
    with open('endpoint_priority_review.json', 'r') as f:
        config = json.load(f)
    
    unknown_endpoints = []
    for endpoint_name, endpoint_info in config['endpoints'].items():
        if endpoint_info['priority'] == 'unknown':
            unknown_endpoints.append({
                'name': endpoint_name,
                'required_params': endpoint_info['required_params']
            })
    
    return unknown_endpoints

def setup_endpoint_for_testing(endpoint_name, required_params):
    """
    Setup an endpoint for testing by creating the import statement and 
    parameter mapping needed for manual testing
    """
    # Build the params dict using only required params
    test_params = {}
    for param in required_params:
        if param in SAMPLE_PARAMS:
            test_params[param] = SAMPLE_PARAMS[param]
        else:
            test_params[param] = f"NEED_VALUE_FOR_{param.upper()}"
    
    # Create the code template
    code_template = f"""
# Testing {endpoint_name}
from nba_api.stats.endpoints import {endpoint_name}

# Create instance with parameters
{endpoint_name.lower()}_instance = {endpoint_name}({', '.join([f"{k}='{v}'" for k, v in test_params.items()])})

# Get dataframes (try index 0 first, then 1, 2, etc. if needed)
try:
    df = {endpoint_name.lower()}_instance.get_data_frames()[0]
    print(f"Shape: {{df.shape}}")
    print(f"Columns: {{list(df.columns)}}")
    print("First few rows:")
    display(df.head())
except Exception as e:
    print(f"Error with index 0: {{e}}")
    print("Available datasets:", len({endpoint_name.lower()}_instance.get_data_frames()))
"""
    
    return {
        'endpoint_name': endpoint_name,
        'required_params': required_params,
        'test_params': test_params,
        'code_template': code_template
    }

# Load unknown endpoints
unknown_endpoints = get_unknown_endpoints()
print(f"Found {len(unknown_endpoints)} unknown endpoints:")
for i, endpoint in enumerate(unknown_endpoints[:10]):  # Show first 10
    print(f"{i+1}. {endpoint['name']} - requires: {endpoint['required_params']}")

if len(unknown_endpoints) > 10:
    print(f"... and {len(unknown_endpoints) - 10} more")

Found 64 unknown endpoints:
1. GameRotation - requires: ['game_id']
2. HustleStatsBoxScore - requires: ['game_id']
3. PlayerGameStreakFinder - requires: []
4. PlayerNextNGames - requires: ['player_id']
5. TeamGameLog - requires: ['team_id']
6. TeamGameLogs - requires: []
7. TeamGameStreakFinder - requires: []
8. CumeStatsPlayer - requires: ['player_id', 'game_ids']
9. FranchisePlayers - requires: ['team_id']
10. InfographicFanDuelPlayer - requires: ['game_id']
... and 54 more


In [13]:
import json
from collections import defaultdict, Counter
import pandas as pd

# Load the full endpoint configuration
with open('endpoint_priority_review.json', 'r') as f:
    endpoint_config = json.load(f)

def categorize_endpoints_by_params():
    """Categorize all endpoints by their required parameters"""
    
    # Group endpoints by required parameters pattern
    param_groups = defaultdict(list)
    param_combinations = Counter()
    
    # Collect all unique parameters
    all_params = set()
    
    for endpoint_name, endpoint_info in endpoint_config['endpoints'].items():
        required_params = endpoint_info['required_params']
        all_params.update(required_params)
        
        # Create a sorted tuple for consistent grouping
        param_key = tuple(sorted(required_params))
        param_groups[param_key].append(endpoint_name)
        param_combinations[param_key] += 1
    
    return param_groups, param_combinations, all_params

# Run the analysis
param_groups, param_combinations, all_params = categorize_endpoints_by_params()

# print("=== ENDPOINT CATEGORIZATION BY REQUIRED PARAMETERS ===\n")

# print(f"Total endpoints: {len(endpoint_config['endpoints'])}")
# print(f"Unique parameter combinations: {len(param_combinations)}")
# print(f"All unique parameters found: {sorted(all_params)}\n")

# # Show parameter combination statistics
# print("=== PARAMETER COMBINATION FREQUENCY ===")
# for params, count in param_combinations.most_common():
#     if not params:  # No parameters required
#         param_str = "NO PARAMETERS"
#     else:
#         param_str = ", ".join(params)
#     print(f"{count:2d} endpoints require: [{param_str}]")

# print("\n" + "="*80)

param_groups

defaultdict(list,
            {('game_id',): ['BoxScoreAdvancedV2',
              'BoxScoreAdvancedV3',
              'BoxScoreDefensiveV2',
              'BoxScoreFourFactorsV2',
              'BoxScoreFourFactorsV3',
              'BoxScoreHustleV2',
              'BoxScoreMatchupsV3',
              'BoxScoreMiscV2',
              'BoxScoreMiscV3',
              'BoxScorePlayerTrackV2',
              'BoxScorePlayerTrackV3',
              'BoxScoreScoringV2',
              'BoxScoreScoringV3',
              'BoxScoreSummaryV2',
              'BoxScoreTraditionalV2',
              'BoxScoreTraditionalV3',
              'BoxScoreUsageV2',
              'BoxScoreUsageV3',
              'GameRotation',
              'HustleStatsBoxScore',
              'PlayByPlay',
              'PlayByPlayV2',
              'PlayByPlayV3',
              'InfographicFanDuelPlayer',
              'VideoEvents',
              'WinProbabilityPBP'],
             ('person1_id', 'person2_id'): ['BoxScoreSimil

In [ ]:
# Import all endpoints at module level
from nba_api.stats import endpoints

# Create a generator for testing endpoints one by one
def endpoint_generator():
    """Generator that yields one endpoint at a time for testing"""
    for param_tuple, endpoint_list in param_groups.items():
        for endpoint_name in endpoint_list:
            yield param_tuple, endpoint_name

# Initialize the generator
endpoint_iter = endpoint_generator()

def test_next_endpoint():
    """Test the next endpoint and return the dataframe"""
    try:
        param_tuple, endpoint_name = next(endpoint_iter)
        
        print(f"Testing: {endpoint_name}")
        print(f"Required params: {', '.join(param_tuple) if param_tuple else 'None'}")
        
        # Get the endpoint class using getattr
        endpoint_class = getattr(endpoints, endpoint_name)
        
        # Build parameters
        kwargs = {}
        missing_params = []
        
        for param in param_tuple:
            if param in SAMPLE_PARAMS:
                kwargs[param] = SAMPLE_PARAMS[param]
            else:
                missing_params.append(param)
        
        if missing_params:
            print(f"Skipping - need values for: {missing_params}")
            return None
        
        # Create instance and get data
        instance = endpoint_class(**kwargs)
        dataframes = instance.get_data_frames()
        
        if dataframes:
            df = dataframes[0]
            print(f"Shape: {df.shape}")
            print(f"Columns: {list(df.columns)}")
            return df
        else:
            print("No dataframes returned")
            return None
            
    except StopIteration:
        print("All endpoints tested!")
        return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Test the first endpoint
df = test_next_endpoint()
if df is not None:
    display(df.head())

Testing: BoxScoreAdvancedV2
Required params: game_id
Error: 'BoxScoreAdvancedV2'
